In [2]:
import numpy as np
import pandas as pd
import random

#РАСПРЕД ПО СПИСКУ МАГОВ
#список складов для распреда
shops = pd.read_excel('C:\анаконда\shop_list.xlsx', names=['КодСклада'],dtype='int')
shop_lst = shops.КодСклада.tolist()

prih1 = pd.read_excel(r'C:\рушники\9215.xlsx')
prih1 = prih1.sort_values(by='Товар')

# словарь тов - количество и список товов
tov_list = prih1.Товар.unique().tolist()
tov_dict = prih1.Товар.value_counts()[prih1.Товар.unique()].to_dict()

# остатки
if prih1.iloc[0, 3] == 'КОЛЬЦО ОБРУЧАЛЬНОЕ':
    stock = pd.read_excel(r'C:\Остатки\Обручальные кольца.xlsx', usecols=['КодСклада','ТоварНо'])
else:
    stock = pd.read_excel(r'C:\Остатки\Товы.xlsx', usecols=['КодСклада','ТоварНо'])
    
stock = stock[stock['КодСклада'].isin(shop_lst) & stock['ТоварНо'].isin(tov_list)]

# таблица остатки товов по складам
pivot = pd.pivot_table(stock,index='КодСклада', columns='ТоварНо', aggfunc=len, fill_value=0).reset_index()
pivot = shops.merge(pivot, how='left')

# добавим в таблицу колонки с нулями по товам, которых не хватает
add_cols =  [x for x in tov_list if x not in pivot.columns.to_list()]
add_zeroes = [list(np.zeros(len(shop_lst), dtype='int'))] * len(add_cols)

pivot = pivot.reindex(columns=pivot.columns.tolist() + add_cols)

pivot.fillna(0, inplace=True)
pivot = pivot.astype('int')
cols = ['КодСклада'] + tov_list
pivot = pivot[cols]

# Корректируем остатки магазинов куда уходит по 3 и по 2 штуки
shops_3 = [5227, 5230]
shops_2 = [5329]

df_3 = pivot[tov_list][pivot['КодСклада'].isin(shops_3)].applymap(lambda x: 0 if (x - 2) < 0 else x - 2)
df_3['КодСклада'] = pivot['КодСклада'][pivot['КодСклада'].isin(shops_3)]

df_2 = pivot[tov_list][pivot['КодСклада'].isin(shops_2)].applymap(lambda x: 0 if (x - 1) < 0 else x - 1)
df_2['КодСклада'] = pivot['КодСклада'][pivot['КодСклада'].isin(shops_2)]

pivot.loc[pivot['КодСклада'].isin(shops_3 + shops_2)] = pd.concat([df_3, df_2])

# Если количество по тову в прихе больше чем в списке то к списку добавить нули, если нет то обрезать
sklad_list = [
    pivot[pivot[x] == 0]['КодСклада'].tolist() + 
    list(np.zeros((tov_dict[x] - len(pivot[pivot[x] == 0]['КодСклада'].tolist())), dtype='int')) 
    if 
    tov_dict[x] > len(pivot[pivot[x] == 0]['КодСклада'].tolist()) 
    else pivot[pivot[x] == 0]['КодСклада'].tolist()[:tov_dict[x]] 
    for x in tov_list
]

sklad_list = [item for sublist in sklad_list for item in sublist]

prih1['Период реализации'] = sklad_list

#prih1 = prih1[prih1['Период реализации'] > 0]

# Change 0 to 7171
prih1['Период реализации'] = prih1['Период реализации'].apply(lambda x: 7171 if x == 0 else x)


# Сохраняем в ексель на лист "движение товара"

name = 'C:/рушники/' + str(random.randrange(5000,10000)) + '.xlsx'
prih1.to_excel(name, sheet_name='Движение товара', index=False)

In [5]:
shops = pd.read_excel('C:\анаконда\shop_list.xlsx', names=['КодСклада'],dtype='int')
shop_lst = shops.КодСклада.tolist()

prih1 = pd.read_excel(r'C:\рушники\КОЛЬЦО.xlsx')
prih1 = prih1.sort_values(by='Товар')

In [8]:
import numpy as np
import pandas as pd

#РАСПРЕД ПО СПИСКУ МАГОВ ПО АРТИКУЛУ
# prih1 = pd.read_excel(r'C:\Users\Dotsenko.Semen\Documents\Книга1.xlsx')
# del prih1['№ Поставщика']
#ИЛИ
prih1 = pd.read_excel(r'C:\рушники\8169.xlsx')
prih1 = prih1.sort_values(by='Товар')
shops = pd.read_excel('C:\анаконда\shop_list.xlsx', names=['КодСклада'],dtype='int')

# словарь тов - количество и список товов
tov_list = prih1.Артикул.unique().tolist()
tov_dict = prih1.Артикул.value_counts()[prih1.Артикул.unique()].to_dict()

# остатки
if prih1.iloc[0, 3] == 'КОЛЬЦО ОБРУЧАЛЬНОЕ':
    stock = pd.read_excel(r'C:\Остатки\Обручальные кольца.xlsx', usecols=['КодСклада','ТоварНо'])
else:
    stock = pd.read_excel(r'C:\Остатки\остатки.xlsx', usecols=['КодСклада','ПоставщикАрт'])
#stock = pd.read_excel(r'C:\Остатки\обручи.xlsx', usecols=['КодСклада','ТоварНо'])
stock = stock[stock['КодСклада'].isin(shops.КодСклада.tolist()) & stock['ПоставщикАрт'].isin(tov_list)]

# таблица остатки товов по складам
pivot = pd.pivot_table(stock,index='КодСклада', columns='ПоставщикАрт', aggfunc=len, fill_value=0).reset_index()
pivot = shops.merge(pivot, how='left')

# добавим в таблицу колонки с нулями по товам, которых не хватает
add_cols =  [x for x in tov_list if x not in pivot.columns.to_list()]
add_zeroes = [list(np.zeros(len(all_llist), dtype='int'))] * len(add_cols)

pivot = pivot.reindex(columns=pivot.columns.tolist() + add_cols)

pivot.fillna(0, inplace=True)
pivot = pivot.astype('int')
pivot = pivot[['КодСклада'] + tov_list]

# Если количество по тову в прихе больше чем в списке то к списку добавить нули, если нет то обрезать
sklad_list = [
    pivot[pivot[x] == 0]['КодСклада'].tolist() + 
    list(np.zeros((tov_dict[x] - len(pivot[pivot[x] == 0]['КодСклада'].tolist())), dtype='int')) 
    if 
    tov_dict[x] > len(pivot[pivot[x] == 0]['КодСклада'].tolist()) 
    else pivot[pivot[x] == 0]['КодСклада'].tolist()[:tov_dict[x]] 
    for x in tov_list
]

sklad_list = [item for sublist in sklad_list for item in sublist]

prih1['Период реализации'] = sklad_list

#prih1 = prih1[prih1['Период реализации'] > 0]

# Change 0 to 7171
#prih1['Период реализации'] = prih1['Период реализации'].apply(lambda x: 7171 if x == 0 else x)


# Сохраняем в ексель на лист "движение товара"

import random
name = 'C:/рушники/' + str(random.randrange(5000,10000)) + '.xlsx'
prih1.to_excel(name, sheet_name='Движение товара', index=False)

ValueError: Usecols do not match columns, columns expected but not found: ['ПоставщикАрт']

In [17]:
prih1[['Период реализации', 'Склад', 'Тип изделия 1']].groupby(by=['Период реализации'], sort=False).count()


,Склад,Тип изделия 1
Период реализации,,
5285,10,10
5155,10,10
4992,10,10
5197,9,9
4345,6,6
5292,8,8
4891,9,9
5034,6,6
4949,4,4


In [4]:
import math as m
m.log(9, 3)

2.0